In [18]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
import pandas as pd
import numpy as np

# own Modules 
from anomaly_classifier import AnomalyClassifier

# MLE Mode: 

In [19]:
mode = "MLE"
threshold = "2sigma"

# MSE Mode:

In [62]:
mode = "MSE"
threshold = "max_minus_{}".format("60percent")

## Parameters phm data

In [63]:
param = {
    "data" : {
        "path" : '../../data/phm_data_challenge/recipe/dataset_for_each_recipe/test/test_recipe_67_labeled.csv',
        "droped_feature" : ["stage", "Lot", "runnum", "recipe", "recipe_step",
                            "up time", "ongoing time", 
                            "ETCHSOURCEUSAGE", "ETCHAUXSOURCETIMER", 
                            "ETCHAUX2SOURCETIMER", "FIXTURESHUTTERPOSITION", "ROTATIONSPEED", "status"
                           ],
        "features_not_to_scale": []
    },
    "model" : {
        "input_size" : 12,
    },
        "anomaly_detection" : {
        "threshold_quadratic_difference" : 0.5,
        "threshold_q_d_each_sensor" : [3.561851, 3.561851, 3.561851, 3.561851, 
                                      3.561851, 3.561851, 3.561851, 3.561851, 
                                       3.561851, 3.561851, 3.561851, 3.561851],
        "no_standard_deviation" : 2
    },
    "prediction_path": "../visualisation/files/prediction/{}/phm_67.csv".format(mode),
    "classification_path" : "../visualisation/files/classification/{}/phm67_{}.csv".format(mode, threshold)
}

## Parameters artifical data

In [39]:
param = {
    "data" : {
        "path" : '../../data/artifical_signals/artifical_2_signals_errors.csv',
        "droped_feature" : ["anomaly"
                           ],
        "features_not_to_scale": []
    },
    "model" : {
        "input_size" : 2,
    },
        "anomaly_detection" : {
        "threshold_quadratic_difference" : 0.5,
        "threshold_q_d_each_sensor" : [0.7012361884117126, 3.1871493458747864], # [0.13679444326055645, 0.5441877437706729],
        "no_standard_deviation" : 2
    },
    "prediction_path": "../visualisation/files/prediction/{}/artfic.csv".format(mode),
    "classification_path" : "../visualisation/files/classification/{}/artfic_{}.csv".format(mode, threshold)
}

## Parameters cpps data

In [14]:
param = {
    "data" : {
        "path" : '../../data/cpps_degradation_new/data_obs10/test/obs_space_error_sinusiod_preprocessed.csv',
        "droped_feature" : ["status"
                           ],
        "features_not_to_scale": []
    },
    "model" : {
        "input_size" : 10,
    },
        "anomaly_detection" : {
        "threshold_quadratic_difference" : 0.5,
        "threshold_q_d_each_sensor" : [0.24372421389777016, 0.24761591192911026, 0.32903928384871034, 
                                       0.18159441767379658, 0.27133063146032477, 0.23155811781422755, 
                                       0.149805263381253, 0.31187742832928145, 0.0716386888894709, 0.2080102174959789],
        "no_standard_deviation" : 1
    },
    "prediction_path": "../visualisation/files/prediction/{}/cpps.csv".format(mode),
    "classification_path" : "../visualisation/files/classification/{}/cpps_{}.csv".format(mode, threshold)
}

In [48]:
all_data = pd.read_csv(param["prediction_path"], sep=";")
training = all_data.iloc[:50000,:]
training.shape[0]

50000

In [49]:
training.max()

ID                                               96186.000000
IONGAUGEPRESSURE target                              1.026571
ETCHBEAMVOLTAGE target                               1.347060
ETCHBEAMCURRENT target                               2.693076
ETCHSUPPRESSORVOLTAGE target                         1.499655
ETCHSUPPRESSORCURRENT target                         3.716567
FLOWCOOLFLOWRATE target                              1.080096
FLOWCOOLPRESSURE target                              0.708334
ETCHGASCHANNEL1READBACK target                       2.066042
ETCHPBNGASREADBACK target                            1.226196
FIXTURETILTANGLE target                              2.617388
ACTUALROTATIONANGLE target                       18040.533203
ACTUALSTEPDURATION target                            3.798495
IONGAUGEPRESSURE predicted                           1.079630
ETCHBEAMVOLTAGE predicted                            1.495523
ETCHBEAMCURRENT predicted                            1.020044
ETCHSUPP

## Detect Anomaly
### Euclidean Distance Metric for the Hole Machine (for MSE Loss Function)

In [34]:
results_prediction = pd.read_csv(param["prediction_path"], sep=";")
anomaly_classifier = AnomalyClassifier()
results = anomaly_classifier.apply_euclidean_distance_metric(results_prediction, 
                                                           param['model']['input_size'], 
                                                           param["anomaly_detection"]["threshold_quadratic_difference"])

### Euclidean Distance Difference Metric for Each Sensor (for MSE Loss Function)

In [64]:
results_prediction = pd.read_csv(param["prediction_path"], sep=";")
anomaly_classifier = AnomalyClassifier()
results = anomaly_classifier.apply_euclidean_distance_metric(results_prediction, 
                                                           param['model']['input_size'],
                                                           param["anomaly_detection"]["threshold_quadratic_difference"],
                                                           percentage = 0.4,
                                                           each_sensor=True, 
                                                           threshold_sensors=param["anomaly_detection"]["threshold_q_d_each_sensor"])

### Standard Deviation Metric (for MLE Loss Function)

In [22]:
results_prediction = pd.read_csv(param["prediction_path"], sep=";")
anomaly_classifier = AnomalyClassifier()
results = anomaly_classifier.apply_prediction_interval_metric(results_prediction, 
                                                            param['model']['input_size'],
                                                            param["anomaly_detection"]["no_standard_deviation"])

In [23]:
results.head()

,ID,IONGAUGEPRESSURE target,ETCHBEAMVOLTAGE target,ETCHBEAMCURRENT target,ETCHSUPPRESSORVOLTAGE target,ETCHSUPPRESSORCURRENT target,FLOWCOOLFLOWRATE target,FLOWCOOLPRESSURE target,ETCHGASCHANNEL1READBACK target,ETCHPBNGASREADBACK target,...,Anomaly Sensor_3,Anomaly Sensor_4,Anomaly Sensor_5,Anomaly Sensor_6,Anomaly Sensor_7,Anomaly Sensor_8,Anomaly Sensor_9,Anomaly Sensor_10,Anomaly Sensor_11,Anomaly Sensor_12
0,22.0,-4.295414,-1.581093,-1.587199,-1.583969,-1.480392,-0.553929,-0.536234,-2.374155,-4.994313,...,0,0,0,0,0,0,0,0,0,0
1,23.0,-4.295414,-1.581425,-1.587977,-1.584024,-1.480392,-0.552278,-0.513811,-2.374155,-4.994313,...,0,0,0,0,0,0,0,0,0,0
2,24.0,-4.296095,-1.581558,-1.588210,-1.584133,-1.480392,-0.552278,-0.500999,-2.374155,-4.992283,...,0,0,0,0,0,0,0,0,0,0
3,25.0,-4.296095,-1.581225,-1.587743,-1.584187,-1.480392,-0.552278,-0.481773,-2.372523,-4.992283,...,0,0,0,0,0,0,0,0,0,0
4,26.0,-4.296739,-1.581358,-1.587899,-1.584296,-1.480392,-0.558423,-0.449735,-2.372523,-4.992283,...,0,0,0,0,0,0,0,0,0,0


## Combine prediction data with data which was not consider for inference

In [65]:
original_sensor_data = pd.read_csv(param["data"]["path"])
data_of_droped_feature = original_sensor_data.loc[:, param["data"]["droped_feature"]+["ID"]]
complete_data = results.merge(right=data_of_droped_feature, how="inner", on="ID")

## Save data to csv file

In [66]:
complete_data.to_csv(param["classification_path"], sep=";", index=False)